### Turn Cytoscape (v 3.8.2) cluster output into binary fasta

#### First run Snakefile_final rule __conserved_domains__ to make the "{sample}\_{sample2}\_orth\_orIDs.txt" files.  
#### Second concatenate (cat) all of these into `all_orthIDs.txt`
Then input into __Cytoscape__.

In [ ]:
#cat -n louvain_all_orthIDs.csv | tail -1455| sed -r 's/"//g' - | sed -r 's/,/\t/g' - | sed -r 's/d/\t d/g' - > louvain_t_all

In [12]:
species = ["dmel", "dsim", "dtak", "deug", "dere", "dyak", "dbia", "dele", "dfic", "dtri", "dana"]

In [13]:
 spc = [">dmel", ">dsim", ">dtak", ">deug", ">dere", ">dyak", ">dbia", ">dele", ">dfic", ">dtri", ">dana"]

In [14]:
import numpy as np

In [15]:
import sys

In [16]:
import pandas as pd

In [17]:
import os

In [ ]:
#in shell -- see below for commands to create dmel_TAD_coords.txt
bedtools intersect -wo -a dmel_TAD_coords.txt_t -b Kc_chromatin_states_r6_muller.bed | sort -k1,1b -k2,2n -k3,3n -k8,8b -k9,9n | bedtools groupby -g 1,2,3,4,8 -c 9 -o sum |sort -k1,1b -k2,2n -k3,3n -k6,6n| bedtools groupby -g 1,2,3,4 -c 5 -o last> dmel_TADs_chromatin.txt

In [24]:
#to create chromatin dictionary - FINAL
f = open("louvain_t_all", "r")
z = open("dmel_TADs_chromatin.txt", "r")
dc = {}
for line in z:
    l = line.split("\t")
    domain = str.strip(l[3])
    did = l[0] + "_" + domain
    dc[did] = l[4][0:3]

In [25]:
#presence absence
f = open("louvain_t_all", "r")
z = open("presense_absence_by_line.txt", "w")
r = open("TAD_ranks_chromatin.txt", "w")
array = []
lc = -1
for line in f:
    msa = {}
    l = line.split("\t")
    #next for loop to find chromatin enrichment of TAD: run "create chromatin dictionary" first 
    for x in l:
        n = x.strip()
        if n[0:4] == "dmel":
            dmid = str.strip(x)
            chromatin = dc.get(dmid)
    #print(dmid + str(chromatin))
    temp = []
    for i in l:
        s = i.strip()
        temp.append(s[0:4])
    #print(temp)
    for sp in species:
        if sp in temp:
            msa[sp] = 1
        else:
            msa[sp]= 0
    line_array = []
    for k,v in msa.items():  
        line_array.append(v)
        z.write(str(k) + ":" + str(v) + "\t")
    #print(str(dmid) + "\t"+ str(chromatin))
    array.append(line_array)
    #following 7 lines to keep track of TAD rank and chromatin
    lc += 1
    count = line_array.count(1)
    if "dmel" in temp:
        r.write(str(lc) + "\t" + str(count) + "\t" + str(chromatin) + "\n")
    else:
        r.write(str(lc) + "\t" + str(count) + "\t" + "no_dmel" + "\n")
    z.write("\n")
z.close()
r.close()

Run the following to split TADs into BLUE/BLACK enriched or YELLOW enriched only. Comment out lines 3 and 18 and comment in lines 4 and 17 to switch colors. 

In [32]:
#presence absence
f = open("louvain_t_all", "r")
#z = open("presense_absence_by_line_yel.txt", "w")
z = open("presense_absence_by_line_blu_bla.txt", "w")
array = []
lc = -1
dm = 0
for line in f:
    msa = {}
    l = line.split("\t")
    #next for loop to find chromatin enrichment of TAD: run "create chromatin dictionary" first 
    for x in l:
        n = x.strip()
        if n[0:4] == "dmel":
            dmid = str.strip(x)
            chromatin = dc.get(dmid)
    if chromatin == "BLU" or chromatin == "BLA":
    #if chromatin == "YEL":
        #print(chromatin)
        temp = []
        for i in l:
            s = i.strip()
            temp.append(s[0:4])
        if "dmel" in temp:
                for sp in species:
                    if sp in temp:
                        msa[sp] = 1
                    else:
                        msa[sp]= 0
                line_array = []
                for k,v in msa.items():  
                    line_array.append(v)
                    z.write(str(k) + ":" + str(v) + "\t")
                #print(str(dmid) + "\t"+ str(chromatin))
                array.append(line_array)
                z.write("\n")
    #following 7 lines to keep track of TAD rank and chromatin
    lc += 1
    count = line_array.count(1)
    #if "dmel" in temp:
    #    r.write(str(lc) + "\t" + str(count) + "\t" + str(chromatin) + "\n")
    #else:
    #    r.write(str(lc) + "\t" + str(count) + "\t" + "no_dmel" + "\n")
z.close()
#r.close()

In [39]:
#following 3 cells to turn presence absense matrix to binary fasta

In [33]:
a = np.asarray(array)
b = a.transpose()
df = pd.DataFrame(b)

In [34]:
#for fasta format
df.insert(loc=0, column = 'sp', value=spc)
#df.to_csv("tad_phylo.txt",index=False, sep =' ')
#df.to_csv("tad_phylo_yel.txt",index=False, sep =' ')
df.to_csv("tad_phylo_blue_black.txt",index=False, sep =' ')

### Use IQ-TREE to create phylogenetic tree from boundary fasta

IQ-TREE v. 2.1.4

In [ ]:
iqtree -s tad_phylo.fasta -pre tad_phylo_gtr2foi -g suvorov_sub_nv.newick -m MFP -o dana
#rerun with the model selection from first command
iqtree -s tad_phylo.fasta -pre tad_phylo_gtr2foi -g suvorov_sub_nv.newick -redo -m GTR2+FO+I -o dana
#to run with bootstrapping
iqtree -s tad_phylo.fasta -pre tad_phylo_gtr2foi -g suvorov_sub_nv.newick -m MFP -o dana -redo -bo 100

To run bootstrap replicates to create trees using only BLUE/BLACK enriched TADs and YELLOW enriched TADs

In [ ]:
iqtree -s tad_phylo_yel.fasta -pre tad_phylo_gtr2foi_yel -g suvorov_sub_nv.newick -redo -m GTR2+FO+I -o dana -bo 100
iqtree -s tad_phylo_blue_black.fasta -pre tad_phylo_gtr2foi_bb -g suvorov_sub_nv.newick -redo -m GTR2+FO+I -o dana -bo 100

suvorov_sub_nv.newick

In [ ]:
(dana,(dtri,((((dbia,dtak),(((dere,dyak),(dsim,dmel)),deug)),dele),dfic)));

### Use the following R script to find boundary evolution per mya 

In [ ]:
Use this script first to subset the file _schemeA.newick_ from https://github.com/SchriderLab/drosophila_phylogeny/tree/master/tree

In [ ]:
library(ape)
library(treeio)
library(tibble)
library(dplyr)
library(ggtree)
filter suvorov tree to have only 11 species#sat<-read.newick("schemeA.newick")
subtre<-tree_subset(sat,node="D_melanogaster", levels_back=8)
subtre_td<-as.treedata(subtre)
to_drop<-c("D_parapallidosa","D_parabipectinata","D_bipectinata","D_parabipectinata","D_m_pallens","D_malerkotliana_malerkotliana","D_p_nigrens","D_pseudoananassae_pseudoananassae","D_ercepeae","D_asahinai","D_lacteicornis","D_rufa","D_tani","D_aurauria","D_pectinifera","D_bakoue","D_burlai","D_sp_aff_chauvacae","D_bocqueti","D_nikananu","D_seguyi","D_vulcana","D_jambulina","D_birchii","D_mayri","D_truncata","D_bunnanda","D_serrata","D_punjabiensis","D_watanabei","D_kikkawai","D_leontia","D_kanapiae","D_subpulchrella","D_suzukii","D_orena","D_teissieri_273_3","D_mauritiana","D_sechellia","D_carrolli","D_rhopaloa","D_kurseongensis","D_fuyamai")
subtest<-drop.tip(subtre,to_drop)
write.nexus(subtest, file="suvorov_sub.nexus", translate=TRUE)

In [ ]:
s<-read.newick("suvorov_sub_renamed.newick")
std<-as.treedata(s)
d<-read.newick("tad_phylo_gtr2foi.treefile")
dtdr <- reroot(d,4)
std<-as.treedata(s)
dtd<-as.treedata(dtdr)
stdt<-as_tibble(std)
dtdt<-as_tibble(dtdr)
stdtc<-mutate(stdt,tmrca = branch.length)
dtdtc<-mutate(dtdt,d_subs = branch.length)
stdtct<-as.treedata(stdtc)
dtdtct<-as.treedata(dtdtc)
merge_all<-merge_tree(stdtct, dtdtct)
mta<-as_tibble(merge_all)
write.jtree(merge)
mta<-mutate(mta,d_subs_per_my=(d_subs*1455)/tmrca)
write.table(mta, file="final_rate_table.txt")

R script for bootstrap trees

In [ ]:
library(ape)
library(treeio)
library(tibble)
library(dplyr)
r<-readLines("trees.boottrees")
for (n in r){
d<-read.newick(text=n)
dtd<-as.treedata(d)
dtdt<-as_tibble(dtd)
dtdtc<-mutate(dtdt,d_subs = branch.length)
dtdtct<-as.treedata(dtdtc)
r<-root(dtdtct, outgroup="dana")
dtd<-as.treedata(r)
dtdt<-as_tibble(dtd)
dtdtc<-mutate(dtdt,d_subs = branch.length)
dtdtct<-as.treedata(dtdtc)
mta<-as_tibble(dtdtct)
mta[is.na(mta)]<-0
subs_sum<-sum(mta$d_subs)
rate<-(subs_sum/156.513)
write(subs_sum, file="subs_sums.txt", append=TRUE)
write(rate, file="subs_rates.txt", append=TRUE)}

### Run the following script to create _domain_by_rank.txt_ for `Snakefile_ME`

In [7]:
f = open("louvain_t_all", "r")
r = open("domain_by_rank.txt", "w")
array = []
lc = -1
for line in f:
    msa = {}
    l = line.split("\t")
    #next for loop to find chromatin enrichment of TAD: run "create chromatin dictionary" first 
    for x in l:
        n = x.strip()
        if n[0:4] == "dmel":
            dmid = str.strip(x)
            #print(n)
    #print(dmid + str(chromatin))
    temp = []
    domain_ids = []
    for i in l:
        s = i.strip()cat 
        temp.append(s[0:4])
        if s[0:4] in species:
            domain_ids.append(s)
    print(temp)
    print(domain_ids)
    for sp in species:
        if sp in temp:
            msa[sp] = 1
        else:
            msa[sp]= 0
    line_array = []
    for k,v in msa.items():  
        line_array.append(v)
        #print(str(k) + ":" + str(v) + "\t")
    #print(str(dmid) + "\t"+ str(chromatin))
    array.append(line_array)
    #following 7 lines to keep track of TAD rank and chromatin
    lc += 1
    count = line_array.count(1)
    #print(temp)
    for i in domain_ids:
        r.write(str(i) + "\t" + str(count) + "\n")
r.close()

['3', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dana', 'dbia', 'dbia', 'dele', 'dele', 'dere', 'dere', 'deug', 'deug', 'dfic', 'dfic', 'dmel', 'dmel', 'dsim', 'dsim', 'dtak', 'dtri', 'dtri', 'dyak', 'dyak', '4.39', '21', 'C432', 'fals', 'C432']
['dana_Muller_A_DOMAINS_1274', 'dana_Muller_A_DOMAINS_58', 'dbia_Muller_A_DOMAINS_1001', 'dbia_Muller_A_DOMAINS_628', 'dele_Muller_A_DOMAINS_1010', 'dele_Muller_A_DOMAINS_42', 'dere_Muller_A_DOMAINS_1006', 'dere_Muller_A_DOMAINS_41', 'deug_Muller_A_DOMAINS_1024', 'deug_Muller_A_DOMAINS_59', 'dfic_Muller_A_DOMAINS_1045', 'dfic_Muller_A_DOMAINS_56', 'dmel_Muller_A_DOMAINS_1109', 'dmel_Muller_A_DOMAINS_73', 'dsim_Muller_A_DOMAINS_1071', 'dsim_Muller_A_DOMAINS_77', 'dtak_Muller_A_DOMAINS_1077', 'dtri_Muller_A_DOMAINS_1226', 'dtri_Muller_A_DOMAINS_52', 'dyak_Muller_A_DOMAINS_34', 'dyak_Muller_A_DOMAINS_990']
['4', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dana', 'dbia', 'dbia', 'dele', 'dele', 'dere', 'dere', 'deug', 'deug', 'dfic', 'dfic

['356', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dana', 'dele', 'dele', 'dere', 'dfic', 'dmel', 'dmel', 'dsim', 'dyak', '3.32', '10', 'C432', 'fals', 'C432']
['dana_Muller_A_DOMAINS_1227', 'dana_Muller_A_DOMAINS_17', 'dele_Muller_A_DOMAINS_631', 'dele_Muller_A_DOMAINS_977', 'dere_Muller_A_DOMAINS_50', 'dfic_Muller_A_DOMAINS_50', 'dmel_Muller_A_DOMAINS_1096', 'dmel_Muller_A_DOMAINS_66', 'dsim_Muller_A_DOMAINS_67', 'dyak_Muller_A_DOMAINS_974']
['357', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dbia', 'dele', 'dere', 'deug', 'dmel', 'dsim', 'dtak', 'dtri', 'dyak', '3.32', '10', 'C432', 'fals', 'C432']
['dana_Muller_A_DOMAINS_104', 'dbia_Muller_A_DOMAINS_111', 'dele_Muller_A_DOMAINS_79', 'dere_Muller_A_DOMAINS_403', 'deug_Muller_A_DOMAINS_84', 'dmel_Muller_A_DOMAINS_92', 'dsim_Muller_A_DOMAINS_102', 'dtak_Muller_A_DOMAINS_403', 'dtri_Muller_A_DOMAINS_57', 'dyak_Muller_A_DOMAINS_1065']
['358', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dbia', 'dele', 'dere', 'dere', 'deug',

['dbia_Muller_A_DOMAINS_19', 'deug_Muller_A_DOMAINS_1022', 'deug_Muller_A_DOMAINS_606', 'dmel_Muller_A_DOMAINS_77', 'dsim_Muller_A_DOMAINS_1077']
['824', '', '0.0', '0.0', '0', '', 'fals', '', 'dbia', 'dbia', 'dfic', 'dmel', 'dsim', '2.32', '5', 'C432', 'fals', 'C432']
['dbia_Muller_E_DOMAINS_1485', 'dbia_Muller_E_DOMAINS_531', 'dfic_Muller_E_DOMAINS_506', 'dmel_Muller_E_DOMAINS_954', 'dsim_Muller_E_DOMAINS_943']
['825', '', '0.0', '0.0', '0', '', 'fals', '', 'dbia', 'dfic', 'dmel', 'dtri', 'dtri', '2.32', '5', 'C432', 'fals', 'C432']
['dbia_Muller_E_DOMAINS_517', 'dfic_Muller_E_DOMAINS_921', 'dmel_Muller_E_DOMAINS_1595', 'dtri_Muller_E_DOMAINS_1730', 'dtri_Muller_E_DOMAINS_703']
['826', '', '0.0', '0.0', '0', '', 'fals', '', 'dbia', 'dbia', 'dfic', 'dfic', 'dsim', '2.32', '5', 'C432', 'fals', 'C432']
['dbia_Muller_D_DOMAINS_1326', 'dbia_Muller_D_DOMAINS_386', 'dfic_Muller_D_DOMAINS_1415', 'dfic_Muller_D_DOMAINS_326', 'dsim_Muller_D_DOMAINS_891']
['827', '', '0.0', '0.0', '0', '', 'fal

['deug_Muller_B_DOMAINS_1083', 'dfic_Muller_B_DOMAINS_111']
['1305', '', '0.0', '0.0', '0', '', 'fals', '', 'dele', 'dfic', '1.0', '2', 'C433', 'fals', 'C433']
['dele_Muller_B_DOMAINS_752', 'dfic_Muller_B_DOMAINS_127']
['1306', '', '0.0', '0.0', '0', '', 'fals', '', 'dbia', 'dfic', '1.0', '2', 'C433', 'fals', 'C433']
['dbia_Muller_E_DOMAINS_571', 'dfic_Muller_E_DOMAINS_454']
['1307', '', '0.0', '0.0', '0', '', 'fals', '', 'dbia', 'dfic', '1.0', '2', 'C433', 'fals', 'C433']
['dbia_Muller_E_DOMAINS_1460', 'dfic_Muller_E_DOMAINS_434']
['1308', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dfic', '1.0', '2', 'C433', 'fals', 'C433']
['dana_Muller_B_DOMAINS_169', 'dfic_Muller_B_DOMAINS_165']
['1309', '', '0.0', '0.0', '0', '', 'fals', '', 'dana', 'dfic', '1.0', '2', 'C433', 'fals', 'C433']
['dana_Muller_B_DOMAINS_729', 'dfic_Muller_B_DOMAINS_115']
['1310', '', '0.0', '0.0', '0', '', 'fals', '', 'deug', 'dyak', '1.0', '2', 'C433', 'fals', 'C433']
['deug_Muller_C_DOMAINS_287', 'dyak_Muller_C

### Run the following script to create _dmel_TAD_coords.txt_ for `Snakefile_ME`

_dmel_domains_all_ID.bed_ from rule `domains`in Snakefile_final

In [18]:
dd = {}
all = open("../merged_domains/dmel_domains_all_ID.bed", "r")
for line in all:
    l = line.strip().split("\t")
    c2 = l[3].strip("\n")
    bid = str(l[0]) + "_" + str(c2)
    c_c = str(l[1]) + "_" + str(l[2])
    k = str(l[0])
    dd[bid] = c_c

In [19]:
f = open("louvain_t_all", "r")
z = open("dmel_TAD_coords.txt", "w")
array = []
for line in f:
    msa = {}
    l = line.split("\t")
    temp = []
    for i in l:
        n = i.strip()
        print(n[0:4])
        if n[0:4] == "dmel":
            s = str.strip(n)
            coords = dd.get(s)
            start_stop = coords.split("_")
            m = i.split("_")
            dib = m[0].strip()
            #print(str(i) + "\t" + start_stop[0] + "\t" + start_stop[1] )
            z.write(str(dib) + "_" + str(m[1]) + "_" + str(m[2]) + "\t" + start_stop[0] + "\t" + start_stop[1] + "\t" + str(m[3]) + "_" + str(m[4]) + "\n")
z.close()

['     3', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_B_DOMAINS_130 ', ' dbia_Muller_B_DOMAINS_235 ', ' dele_Muller_B_DOMAINS_1087 ', ' dele_Muller_B_DOMAINS_720 ', ' dere_Muller_B_DOMAINS_1060 ', ' dere_Muller_B_DOMAINS_477 ', ' deug_Muller_B_DOMAINS_1030 ', ' deug_Muller_B_DOMAINS_649 ', ' dfic_Muller_B_DOMAINS_176 ', ' dmel_Muller_B_DOMAINS_1131 ', ' dmel_Muller_B_DOMAINS_722 ', ' dsim_Muller_B_DOMAINS_1088 ', ' dsim_Muller_B_DOMAINS_755 ', ' dtak_Muller_B_DOMAINS_1136 ', ' dtri_Muller_B_DOMAINS_236 ', ' dyak_Muller_B_DOMAINS_1051 ', ' dyak_Muller_B_DOMAINS_534', '4.087', '17', 'C27693', 'false', 'C27693\n']
3

0.0
0.0
0

fals

dana
dbia
dele
dele
dere
dere
deug
deug
dfic
dmel
dmel_Muller_B_DOMAINS_1131
dmel
dmel_Muller_B_DOMAINS_722
dsim
dsim
dtak
dtri
dyak
dyak
4.08
17
C276
fals
C276
['     4', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_A_DOMAINS_1236 ', ' dana_Muller_A_DOMAINS_568 ', ' dbia_Muller_A_DOMAINS_1040 ', ' dbia_Muller_A_DOMAINS_681 ', ' dele_Mulle

dele
dere
deug
deug
dfic
dmel
dmel_Muller_A_DOMAINS_1061
dsim
dtak
dtak
dyak
3.7
13
C277
fals
C277
['    35', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_A_DOMAINS_48 ', ' dere_Muller_A_DOMAINS_37 ', ' deug_Muller_A_DOMAINS_593 ', ' dfic_Muller_A_DOMAINS_81 ', ' dmel_Muller_A_DOMAINS_1072 ', ' dmel_Muller_A_DOMAINS_666 ', ' dsim_Muller_A_DOMAINS_1043 ', ' dsim_Muller_A_DOMAINS_709 ', ' dtak_Muller_A_DOMAINS_978 ', ' dtri_Muller_A_DOMAINS_1235 ', ' dtri_Muller_A_DOMAINS_867 ', ' dyak_Muller_A_DOMAINS_35', '3.585', '12', 'C27725', 'false', 'C27725\n']
35

0.0
0.0
0

fals

dbia
dere
deug
dfic
dmel
dmel_Muller_A_DOMAINS_1072
dmel
dmel_Muller_A_DOMAINS_666
dsim
dsim
dtak
dtri
dtri
dyak
3.58
12
C277
fals
C277
['    36', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_D_DOMAINS_409 ', ' dele_Muller_D_DOMAINS_395 ', ' dere_Muller_D_DOMAINS_210 ', ' deug_Muller_D_DOMAINS_1181 ', ' dfic_Muller_D_DOMAINS_313 ', ' dmel_Muller_D_DOMAINS_1302 ', ' dmel_Muller_D_DOMAINS_879 ', ' dsim_

0.0
0.0
0

fals

dana
dbia
dele
deug
dfic
dfic
dmel
dmel_Muller_D_DOMAINS_1272
dmel
dmel_Muller_D_DOMAINS_846
dsim
dtri
dyak
dyak
3.58
12
C277
fals
C277
['    70', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_A_DOMAINS_28 ', ' dere_Muller_A_DOMAINS_387 ', ' dere_Muller_A_DOMAINS_989 ', ' deug_Muller_A_DOMAINS_604 ', ' deug_Muller_A_DOMAINS_987 ', ' dfic_Muller_A_DOMAINS_1024 ', ' dfic_Muller_A_DOMAINS_597 ', ' dmel_Muller_A_DOMAINS_1060 ', ' dmel_Muller_A_DOMAINS_652 ', ' dsim_Muller_A_DOMAINS_62 ', ' dtak_Muller_A_DOMAINS_1063 ', ' dtak_Muller_A_DOMAINS_437', '3.585', '12', 'C27760', 'false', 'C27760\n']
70

0.0
0.0
0

fals

dbia
dere
dere
deug
deug
dfic
dfic
dmel
dmel_Muller_A_DOMAINS_1060
dmel
dmel_Muller_A_DOMAINS_652
dsim
dtak
dtak
3.58
12
C277
fals
C277
['    71', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_C_DOMAINS_1539 ', ' dana_Muller_C_DOMAINS_842 ', ' dbia_Muller_C_DOMAINS_284 ', ' dele_Muller_C_DOMAINS_230 ', ' dere_Muller_C_DOMAINS_122 ', ' deug_Muller_


dana
dbia
dele
dere
deug
dfic
dmel
dmel_Muller_E_DOMAINS_586
dsim
dtak
dtri
dyak
3.45
11
C278
fals
C278
['   111', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_E_DOMAINS_458 ', ' dbia_Muller_E_DOMAINS_589 ', ' dele_Muller_E_DOMAINS_486 ', ' dere_Muller_E_DOMAINS_293 ', ' deug_Muller_E_DOMAINS_429 ', ' dfic_Muller_E_DOMAINS_487 ', ' dmel_Muller_E_DOMAINS_563 ', ' dsim_Muller_E_DOMAINS_517 ', ' dtak_Muller_E_DOMAINS_334 ', ' dtri_Muller_E_DOMAINS_790 ', ' dyak_Muller_E_DOMAINS_375', '3.459', '11', 'C27801', 'false', 'C27801\n']
111

0.0
0.0
0

fals

dana
dbia
dele
dere
deug
dfic
dmel
dmel_Muller_E_DOMAINS_563
dsim
dtak
dtri
dyak
3.45
11
C278
fals
C278
['   112', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_C_DOMAINS_228 ', ' dbia_Muller_C_DOMAINS_337 ', ' dele_Muller_C_DOMAINS_235 ', ' dere_Muller_C_DOMAINS_151 ', ' deug_Muller_C_DOMAINS_278 ', ' dfic_Muller_C_DOMAINS_218 ', ' dmel_Muller_C_DOMAINS_321 ', ' dsim_Muller_C_DOMAINS_332 ', ' dtak_Muller_C_DOMAINS_177 ', ' 

3.32
10
C278
fals
C278
['   154', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_C_DOMAINS_323 ', ' dele_Muller_C_DOMAINS_262 ', ' dere_Muller_C_DOMAINS_143 ', ' deug_Muller_C_DOMAINS_1095 ', ' dfic_Muller_C_DOMAINS_251 ', ' dmel_Muller_C_DOMAINS_273 ', ' dsim_Muller_C_DOMAINS_288 ', ' dtak_Muller_C_DOMAINS_619 ', ' dtri_Muller_C_DOMAINS_355 ', ' dyak_Muller_C_DOMAINS_182', '3.322', '10', 'C27844', 'false', 'C27844\n']
154

0.0
0.0
0

fals

dbia
dele
dere
deug
dfic
dmel
dmel_Muller_C_DOMAINS_273
dsim
dtak
dtri
dyak
3.32
10
C278
fals
C278
['   155', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_B_DOMAINS_131 ', ' dele_Muller_B_DOMAINS_132 ', ' dere_Muller_B_DOMAINS_1061 ', ' deug_Muller_B_DOMAINS_118 ', ' dfic_Muller_B_DOMAINS_1100 ', ' dfic_Muller_B_DOMAINS_671 ', ' dmel_Muller_B_DOMAINS_157 ', ' dsim_Muller_B_DOMAINS_1087 ', ' dsim_Muller_B_DOMAINS_754 ', ' dtri_Muller_B_DOMAINS_235', '3.322', '10', 'C27845', 'false', 'C27845\n']
155

0.0
0.0
0

fals

dana
dele
dere
deu

...

fals
C288
['  1113', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_C_DOMAINS_274 ', ' dfic_Muller_C_DOMAINS_1178 ', ' dtri_Muller_C_DOMAINS_1359', '1.585', '3', 'C28803', 'false', 'C28803\n']
1113

0.0
0.0
0

fals

dana
dfic
dtri
1.58
3
C288
fals
C288
['  1114', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_C_DOMAINS_272 ', ' dfic_Muller_C_DOMAINS_1148 ', ' dsim_Muller_C_DOMAINS_860', '1.585', '3', 'C28804', 'false', 'C28804\n']
1114

0.0
0.0
0

fals

dana
dfic
dsim
1.58
3
C288
fals
C288
['  1115', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_C_DOMAINS_251 ', ' dere_Muller_C_DOMAINS_1178 ', ' dmel_Muller_C_DOMAINS_1243', '1.585', '3', 'C28805', 'false', 'C28805\n']
1115

0.0
0.0
0

fals

dana
dere
dmel
dmel_Muller_C_DOMAINS_1243
1.58
3
C288
fals
C288
['  1116', '', '0.0', '0.0', '0', '', 'false', '', ' dana_Muller_C_DOMAINS_247 ', ' dere_Muller_C_DOMAINS_152 ', ' deug_Muller_C_DOMAINS_752', '1.585', '3', 'C28806', 'false', 'C28806\n']
1116

0.0
0.0
0

fals

dan

['  1183', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_E_DOMAINS_521 ', ' dtak_Muller_E_DOMAINS_308', '1.0', '2', 'C28873', 'false', 'C28873\n']
1183

0.0
0.0
0

fals

dbia
dtak
1.0
2
C288
fals
C288
['  1184', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_E_DOMAINS_1238 ', ' dtak_Muller_E_DOMAINS_270', '1.0', '2', 'C28874', 'false', 'C28874\n']
1184

0.0
0.0
0

fals

dbia
dtak
1.0
2
C288
fals
C288
['  1185', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_D_DOMAINS_399 ', ' dtak_Muller_D_DOMAINS_236', '1.0', '2', 'C28875', 'false', 'C28875\n']
1185

0.0
0.0
0

fals

dbia
dtak
1.0
2
C288
fals
C288
['  1186', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_D_DOMAINS_809 ', ' dtak_Muller_D_DOMAINS_197', '1.0', '2', 'C28876', 'false', 'C28876\n']
1186

0.0
0.0
0

fals

dbia
dtak
1.0
2
C288
fals
C288
['  1187', '', '0.0', '0.0', '0', '', 'false', '', ' dsim_Muller_D_DOMAINS_395 ', ' dyak_Muller_D_DOMAINS_745', '1.0', '2', 'C28877', 'false', 'C28877\n']
1187

0.

1252

0.0
0.0
0

fals

dfic
dyak
1.0
2
C289
fals
C289
['  1253', '', '0.0', '0.0', '0', '', 'false', '', ' dfic_Muller_A_DOMAINS_25 ', ' dyak_Muller_A_DOMAINS_945', '1.0', '2', 'C28943', 'false', 'C28943\n']
1253

0.0
0.0
0

fals

dfic
dyak
1.0
2
C289
fals
C289
['  1254', '', '0.0', '0.0', '0', '', 'false', '', ' dfic_Muller_A_DOMAINS_78 ', ' dtri_Muller_A_DOMAINS_123', '1.0', '2', 'C28944', 'false', 'C28944\n']
1254

0.0
0.0
0

fals

dfic
dtri
1.0
2
C289
fals
C289
['  1255', '', '0.0', '0.0', '0', '', 'false', '', ' dfic_Muller_E_DOMAINS_514 ', ' dtri_Muller_E_DOMAINS_774', '1.0', '2', 'C28945', 'false', 'C28945\n']
1255

0.0
0.0
0

fals

dfic
dtri
1.0
2
C289
fals
C289
['  1256', '', '0.0', '0.0', '0', '', 'false', '', ' dfic_Muller_A_DOMAINS_46 ', ' dtri_Muller_A_DOMAINS_70', '1.0', '2', 'C28946', 'false', 'C28946\n']
1256

0.0
0.0
0

fals

dfic
dtri
1.0
2
C289
fals
C289
['  1257', '', '0.0', '0.0', '0', '', 'false', '', ' dfic_Muller_C_DOMAINS_246 ', ' dtri_Muller_C_DOMAINS_410', '1

0.0
0.0
0

fals

dere
dsim
1.0
2
C290
fals
C290
['  1325', '', '0.0', '0.0', '0', '', 'false', '', ' dere_Muller_B_DOMAINS_92 ', ' dmel_Muller_B_DOMAINS_161', '1.0', '2', 'C29015', 'false', 'C29015\n']
1325

0.0
0.0
0

fals

dere
dmel
dmel_Muller_B_DOMAINS_161
1.0
2
C290
fals
C290
['  1326', '', '0.0', '0.0', '0', '', 'false', '', ' dere_Muller_B_DOMAINS_69 ', ' dmel_Muller_B_DOMAINS_115', '1.0', '2', 'C29016', 'false', 'C29016\n']
1326

0.0
0.0
0

fals

dere
dmel
dmel_Muller_B_DOMAINS_115
1.0
2
C290
fals
C290
['  1327', '', '0.0', '0.0', '0', '', 'false', '', ' dere_Muller_E_DOMAINS_251 ', ' dmel_Muller_E_DOMAINS_528', '1.0', '2', 'C29017', 'false', 'C29017\n']
1327

0.0
0.0
0

fals

dere
dmel
dmel_Muller_E_DOMAINS_528
1.0
2
C290
fals
C290
['  1328', '', '0.0', '0.0', '0', '', 'false', '', ' dere_Muller_D_DOMAINS_159 ', ' dmel_Muller_D_DOMAINS_340', '1.0', '2', 'C29018', 'false', 'C29018\n']
1328

0.0
0.0
0

fals

dere
dmel
dmel_Muller_D_DOMAINS_340
1.0
2
C290
fals
C290
['  1329', '',

1392

0.0
0.0
0

fals

dbia
dfic
1.0
2
C290
fals
C290
['  1393', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_C_DOMAINS_333 ', ' dfic_Muller_C_DOMAINS_776', '1.0', '2', 'C29083', 'false', 'C29083\n']
1393

0.0
0.0
0

fals

dbia
dfic
1.0
2
C290
fals
C290
['  1394', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_B_DOMAINS_232 ', ' dfic_Muller_B_DOMAINS_164', '1.0', '2', 'C29084', 'false', 'C29084\n']
1394

0.0
0.0
0

fals

dbia
dfic
1.0
2
C290
fals
C290
['  1395', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_A_DOMAINS_37 ', ' deug_Muller_A_DOMAINS_62', '1.0', '2', 'C29085', 'false', 'C29085\n']
1395

0.0
0.0
0

fals

dbia
deug
1.0
2
C290
fals
C290
['  1396', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_C_DOMAINS_292 ', ' deug_Muller_C_DOMAINS_1110', '1.0', '2', 'C29086', 'false', 'C29086\n']
1396

0.0
0.0
0

fals

dbia
deug
1.0
2
C290
fals
C290
['  1397', '', '0.0', '0.0', '0', '', 'false', '', ' dbia_Muller_C_DOMAINS_277 ', ' deug_Muller_C_DOMAINS_726',